In [34]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [118]:
X = load_wine(return_X_y=False, as_frame=True)

In [120]:
X['data'].rename(columns={'od280/od315_of_diluted_wines':'od280'}, inplace=True)

d:\Anaconda\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [122]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X['data'], X['target'])

RandomForestClassifier(random_state=42)

In [124]:
y_pred = rf.predict(X['data'])
print(classification_report(X['target'], y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        59
           1       1.00      1.00      1.00        71
           2       1.00      1.00      1.00        48

    accuracy                           1.00       178
   macro avg       1.00      1.00      1.00       178
weighted avg       1.00      1.00      1.00       178



In [125]:
confusion_matrix(X['target'], y_pred) 

array([[59,  0,  0],
       [ 0, 71,  0],
       [ 0,  0, 48]], dtype=int64)

In [126]:
import pickle

pickle.dump(rf, open('saved_models/model_01.pkl', 'wb'))

pickle.dump(list(X['data'].columns), open('saved_models/columns_01.pkl', 'wb'))


In [79]:
!python api.py

^C


In [194]:
import json
## para testear en postman
final = json.dumps([X['data'].loc[90].to_dict(), X['data'].loc[0].to_dict()], indent=2)
print(final)

[
  {
    "alcohol": 12.08,
    "malic_acid": 1.83,
    "ash": 2.32,
    "alcalinity_of_ash": 18.5,
    "magnesium": 81.0,
    "total_phenols": 1.6,
    "flavanoids": 1.5,
    "nonflavanoid_phenols": 0.52,
    "proanthocyanins": 1.64,
    "color_intensity": 2.4,
    "hue": 1.08,
    "od280": 2.27,
    "proline": 480.0
  },
  {
    "alcohol": 14.23,
    "malic_acid": 1.71,
    "ash": 2.43,
    "alcalinity_of_ash": 15.6,
    "magnesium": 127.0,
    "total_phenols": 2.8,
    "flavanoids": 3.06,
    "nonflavanoid_phenols": 0.28,
    "proanthocyanins": 2.29,
    "color_intensity": 5.64,
    "hue": 1.04,
    "od280": 3.92,
    "proline": 1065.0
  }
]


In [83]:
import requests

In [200]:
response = requests.post("http://127.0.0.1:12345/predict", json=[X['data'].loc[10].to_dict(), X['data'].loc[90].to_dict()])
print(response)
print('prediccion: ', response.json())

<Response [200]>
prediccion:  {'prediction': '[0, 1]'}


In [193]:
import random
import time

for i in range(100):
    time.sleep(random.randint(0, 3))
    response = requests.post("http://127.0.0.1:12345/predict", json=[X['data'].sample(random.randint(0, len(X['data']))).to_dict()])

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=12345): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002A26FB2AFD0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))